# Загрузка "потока событий" в Redis

In [41]:
from typing import Dict, List

import redis
import json

import pandas as pd


class Constants:
    USER_ID = "user_id"
    ITEM_ID = "item_id"
    ITEMS_IDS = "items_ids"
    TIMESTAMP = "time"

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"


def load_data_to_redis(user_sequences: List[Dict], r: redis.Redis) -> None:
    """
    Загрузка данных в Redis
    """
    for user_sequence in user_sequences:
        user_id = user_sequence.get(Constants.USER_ID)
        items_ids = user_sequence.get(Constants.ITEMS_IDS)

        r.set(f"user:{user_id}", json.dumps(items_ids))

    print("Данные успешно загружены в Redis")


# Подключение к Redis
r = redis.Redis(
    host="localhost",
    password="redis123",
    port=6379,
    decode_responses=True,
)

In [42]:
# Загрузка предобработанных транзакций
transactions = pd.read_parquet(Constants.TRANSACTIONS_PATH)

# Подготовка последовательностей для каждого пользователя
user_sequences = (
    transactions.groupby([Constants.USER_ID], as_index=False)[Constants.ITEM_ID]
    .apply(list)
    .rename(columns={Constants.ITEM_ID: Constants.ITEMS_IDS})
    .to_dict("records")
)

# Загрузка данных в Redis
load_data_to_redis(user_sequences=user_sequences, r=r)

Данные успешно загружены в Redis


In [44]:
[u["user_id"] for u in user_sequences]

['02703e242da346fc85f1aad21208d1103e62e705564c81ca2b6edf943032320a',
 '0350db312b33ec58e9b4e40c56998594d613476ec605b4c5a5a435e9ed172d70',
 '03fdb0bf2d9ff8ba23e1b4aef53709119aad5bc83691d89293a01a52b93d7370',
 '04dab48e5805e9c05272604ac78eb5eb941850ce307a7dd4bb5fe4652c0e4915',
 '0501552eaebc51b1199e9f900b5ba7bedf975624787d6bb42601293cb6743395',
 '057dddf8b97b949ae81c5e6f1269027ce573fca569b9c27c004ea42713dd46cb',
 '06878ff9b50fa7b950cb6f592d9d9369f1adc61ca6f62408aded14911cc91556',
 '06d23b72cac134851a761473b40c881c300814981b6b0a3c4692ddf5a93f2a48',
 '0bf4c6fd4e9d33f9bfb807bb78348cbf5c565846ff4006acf5c1b9aea77b0e54',
 '10411ac70c80005a535783ede440e0de5df6a1d7ff25fde5f58ac00ca97d8e55',
 '13592d7c71d4738bf3c5e641a82ea166832a5f92d856b22d5b4d6cebbfbece2f',
 '15a075c3ea7b76dc31971a496c0d4b1f5f30170355eff7cedd4f4234c8c9402c',
 '165850b79c905aec84972ae0bf62c28c4c4c7fcc2265471c688861b0049ce55c',
 '193badf4539ab95c1ac39be4a302e6fe190b3a6a5c8b50465fdbbdbbffdcb050',
 '1b96fab4fb82d31a42b6ea6bc92a2c5b

In [8]:
# Проверка данных для случайного пользователя
user_id = "02703e242da346fc85f1aad21208d1103e62e705564c81ca2b6edf943032320a"
data = r.get(f"user:{user_id}")

if data:
    items = json.loads(data)  # Десериализация из JSON
    print(f"Товары пользователя {user_id}: {items}")
else:
    print("Ключ не найден")

Товары пользователя 02703e242da346fc85f1aad21208d1103e62e705564c81ca2b6edf943032320a: [573716012, 573716012, 568597006, 568597006, 673677002, 673677002, 673677002, 565379002, 685816002, 562245050, 562245050, 399256005, 399256005, 573716012, 573716012, 562245050, 573716012, 573716012, 565379001, 730683001, 158340001, 111593001, 568601006, 568601006, 573716012, 573716012, 568597006, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 507909001, 599580038, 536139006, 684209013, 684209013, 572797002, 572797002, 572797002, 572797001, 572797001]


In [ ]:
import requests

response = requests.get(
    "http://localhost:8000/recommendations/personal_items/",
    params={
        "user_id": "test"
    },
)
response.json()

In [28]:
import requests

response = requests.get(
    "http://localhost:8000/recommendations/product_images/",
    params='article_ids=["156231001", "158340001", "539723005"]',
)
response

<Response [422]>

In [45]:
import requests
import base64
from io import BytesIO
from PIL import Image

# URL вашего API
url = "http://localhost:8000/recommendations/product_images/"

# Список ID изображений, которые нужно получить
article_ids = ["0706016001"]  # пример ID

# Делаем GET-запрос с параметрами
params = {"article_ids": article_ids}
response = requests.get(url, params=params)

if response.status_code == 200:
    images_data = response.json()
    
    for article_id, img_base64 in images_data.items():
        if img_base64:
            # Декодируем base64 в байты
            img_bytes = base64.b64decode(img_base64)
            
            # Создаем изображение из байтов
            img = Image.open(BytesIO(img_bytes))
            img.show()  # или сохраняем: img.save(f"{article_id}.jpg")
            print(f"Successfully received image {article_id}")
        else:
            print(f"Image {article_id} not found")
else:
    print(f"Error: {response.status_code}, {response.text}")

Successfully received image 0706016001


In [ ]:
response.json()

{'0156231001': '...',
 '0158340001': '...',
 '0539723005': '...'}

In [34]:
import boto3
from botocore.client import Config

# Инициализация клиента для MinIO
s3 = boto3.client(
    's3',
    endpoint_url='http://localhost:9000',  # или ваш URL MinIO
    aws_access_key_id="minio",
    aws_secret_access_key="minio123",
    config=Config(signature_version='s3v4'),  # важно для MinIO
)

bucket_name = 'product-images'

def list_s3_objects(bucket):
    try:
        response = s3.list_objects_v2(Bucket=bucket)
        if 'Contents' in response:
            return [obj['Key'] for obj in response['Contents']]
        return []
    except Exception as e:
        print(f"Error listing objects: {e}")
        return []

objects = list_s3_objects(bucket_name)
print(f"Found {len(objects)} objects:")
for obj in objects:
    print(obj)

Found 98 objects:
0108775015.jpg
0111586001.jpg
0111593001.jpg
0156231001.jpg
0158340001.jpg
0160442007.jpg
0160442010.jpg
0179123001.jpg
0228257001.jpg
0351484002.jpg
0372860001.jpg
0372860002.jpg
0399223001.jpg
0399223029.jpg
0399256001.jpg
0399256005.jpg
0436261001.jpg
0448509001.jpg
0448509014.jpg
0464297007.jpg
0484398001.jpg
0507909001.jpg
0507910001.jpg
0536139006.jpg
0537116001.jpg
0539723005.jpg
0554450001.jpg
0554479001.jpg
0554598001.jpg
0562245001.jpg
0562245018.jpg
0562245046.jpg
0562245050.jpg
0564786001.jpg
0565379001.jpg
0565379002.jpg
0568597006.jpg
0568601006.jpg
0572797001.jpg
0572797002.jpg
0573085028.jpg
0573716012.jpg
0579541001.jpg
0590928001.jpg
0599580017.jpg
0599580038.jpg
0599580055.jpg
0600886001.jpg
0608776002.jpg
0611415001.jpg
0624486001.jpg
0636323001.jpg
0673396002.jpg
0673677002.jpg
0678942001.jpg
0684209004.jpg
0684209013.jpg
0685813001.jpg
0685814001.jpg
0685816001.jpg
0685816002.jpg
0688537004.jpg
0688537011.jpg
0689109001.jpg
0695632001.jpg
0695632

In [18]:
import requests
import json

url = "http://localhost:8000/recommendations/product_images_base64/"
payload = ["123456789", "987654321", "010203040"]
headers = {"Content-Type": "application/json"}

response = requests.post(url, data=json.dumps(payload), headers=headers)
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [14]:
response

<Response [500]>